#  PHASE 2: SQL-First Exploratory Data Analysis

## Big-Tech-Grade User Retention & Churn Prediction System

---

**Author**: Senior Data Scientist  
**Date**: February 2026  
**Objective**: Answer critical business questions about churn using SQL-style analysis

---

##  Key Questions to Answer
1. What % of customers churn?
2. How does activity decline before churn?
3. How does purchase frequency differ between churned vs active?
4. What is average time-to-churn?
5. Which behavioral patterns correlate with churn?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

try:
    from pandasql import sqldf
    pysqldf = lambda q: sqldf(q, globals())
    SQL_AVAILABLE = True
except ImportError:
    print("pandasql not available, using pandas operations")
    SQL_AVAILABLE = False

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Libraries imported successfully")

In [ ]:
import os

PROJECT_ROOT = '/Users/anuj/Desktop/Churn_Retension/churn-prediction-bigtech'
PROCESSED_DATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'processed')

df = pd.read_parquet(os.path.join(PROCESSED_DATA_PATH, 'transactions_clean.parquet'))

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print(f"✅ Loaded {len(df):,} transactions")
print(f"   Customers: {df['Customer ID'].nunique():,}")
print(f"   Date range: {df['InvoiceDate'].min().date()} to {df['InvoiceDate'].max().date()}")

In [ ]:

DATA_END_DATE = df['InvoiceDate'].max().date()
OBSERVATION_DATE = pd.Timestamp('2011-10-10')
CHURN_WINDOW = 60

print(f"📅 Key Dates:")
print(f"   Data End Date: {DATA_END_DATE}")
print(f"   Observation Date: {OBSERVATION_DATE.date()}")
print(f"   Churn Window: {CHURN_WINDOW} days")
print(f"   Churn Cutoff: {(OBSERVATION_DATE - timedelta(days=CHURN_WINDOW)).date()}")

---

## 1. What % of Customers Churn?

**Definition**: Churn = No purchase in 60 days after observation date

In [ ]:
customer_summary = df.groupby('Customer ID').agg(
    first_purchase=('InvoiceDate', 'min'),
    last_purchase=('InvoiceDate', 'max'),
    total_orders=('Invoice', 'nunique'),
    total_revenue=('Price', lambda x: (x * df.loc[x.index, 'Quantity']).sum()),
    total_items=('Quantity', 'sum')
).reset_index()

customer_summary['tenure_days'] = (customer_summary['last_purchase'] - customer_summary['first_purchase']).dt.days

churn_cutoff = OBSERVATION_DATE - timedelta(days=CHURN_WINDOW)

customer_summary_filtered = customer_summary[
    customer_summary['first_purchase'] <= churn_cutoff
].copy()

customer_summary_filtered['is_churned'] = (
    customer_summary_filtered['last_purchase'] <= churn_cutoff
).astype(int)

print(f"📊 CHURN RATE ANALYSIS")
print(f"="*50)
print(f"Total customers (eligible): {len(customer_summary_filtered):,}")
print(f"Churned customers: {customer_summary_filtered['is_churned'].sum():,}")
print(f"Active customers: {(1 - customer_summary_filtered['is_churned']).sum():,}")
print(f"\n🎯 CHURN RATE: {customer_summary_filtered['is_churned'].mean()*100:.2f}%")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

churn_counts = customer_summary_filtered['is_churned'].value_counts()
labels = ['Active', 'Churned']
colors = ['#2ecc71', '#e74c3c']
explode = (0, 0.05)

axes[0].pie(churn_counts.values, labels=labels, colors=colors, explode=explode,
            autopct='%1.1f%%', shadow=True, startangle=90)
axes[0].set_title('Customer Churn Distribution', fontsize=14, fontweight='bold')

bars = axes[1].bar(['Active', 'Churned'], 
                    [len(customer_summary_filtered) - churn_counts[1], churn_counts[1]],
                    color=colors)
axes[1].set_ylabel('Number of Customers')
axes[1].set_title('Customer Counts by Status', fontsize=14, fontweight='bold')

for bar in bars:
    height = bar.get_height()
    axes[1].annotate(f'{int(height):,}',
                     xy=(bar.get_x() + bar.get_width() / 2, height),
                     ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
print("\n📊 CHURN RATE BY SEGMENT")
print("="*60)

customer_summary_filtered['order_bucket'] = pd.cut(
    customer_summary_filtered['total_orders'],
    bins=[0, 1, 3, 5, 10, float('inf')],
    labels=['1 order', '2-3 orders', '4-5 orders', '6-10 orders', '10+ orders']
)

churn_by_orders = customer_summary_filtered.groupby('order_bucket').agg(
    customers=('Customer ID', 'count'),
    churned=('is_churned', 'sum'),
    churn_rate=('is_churned', 'mean')
).round(4)
churn_by_orders['churn_rate_pct'] = (churn_by_orders['churn_rate'] * 100).round(2)

print("\nBy Purchase Frequency:")
print(churn_by_orders[['customers', 'churned', 'churn_rate_pct']])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x = range(len(churn_by_orders))
bars = ax.bar(x, churn_by_orders['churn_rate_pct'], color=plt.cm.RdYlGn_r(churn_by_orders['churn_rate_pct']/100))
ax.set_xticks(x)
ax.set_xticklabels(churn_by_orders.index, rotation=0)
ax.set_ylabel('Churn Rate (%)')
ax.set_xlabel('Purchase Frequency')
ax.set_title('Churn Rate by Purchase Frequency', fontsize=14, fontweight='bold')
ax.axhline(y=customer_summary_filtered['is_churned'].mean()*100, color='red', linestyle='--', 
           label=f'Overall: {customer_summary_filtered["is_churned"].mean()*100:.1f}%')

for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}%',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                ha='center', va='bottom', fontsize=10)

ax.legend()
plt.tight_layout()
plt.show()

print("\n💡 INSIGHT: One-time buyers have dramatically higher churn rates!")
print("   → Getting customers to make a 2nd purchase is critical for retention")

---

## 2. How Does Activity Decline Before Churn?

Analyzing the behavioral signals that precede customer churn.

In [ ]:
churned_customers = customer_summary_filtered[customer_summary_filtered['is_churned'] == 1]['Customer ID'].values
active_customers = customer_summary_filtered[customer_summary_filtered['is_churned'] == 0]['Customer ID'].values

print(f"Analyzing {len(churned_customers):,} churned customers")
print(f"Comparing with {len(active_customers):,} active customers")

In [ ]:
def analyze_activity_windows(df, customer_ids, customer_type=''):
    """Analyze purchase activity in windows before last purchase"""
    
    df_subset = df[df['Customer ID'].isin(customer_ids)].copy()
    
    last_purchase = df_subset.groupby('Customer ID')['InvoiceDate'].max().reset_index()
    last_purchase.columns = ['Customer ID', 'last_purchase_date']
    
    df_subset = df_subset.merge(last_purchase, on='Customer ID')
    
    df_subset['days_before_last'] = (df_subset['last_purchase_date'] - df_subset['InvoiceDate']).dt.days
    
    def assign_window(days):
        if days <= 30:
            return 'Last 30 days'
        elif days <= 60:
            return '31-60 days before'
        elif days <= 90:
            return '61-90 days before'
        else:
            return '90+ days before'
    
    df_subset['time_window'] = df_subset['days_before_last'].apply(assign_window)
    
    window_stats = df_subset.groupby('time_window').agg(
        unique_customers=('Customer ID', 'nunique'),
        total_orders=('Invoice', 'nunique'),
        total_revenue=('Price', lambda x: (x * df_subset.loc[x.index, 'Quantity']).sum())
    ).reset_index()
    
    window_stats['orders_per_customer'] = window_stats['total_orders'] / window_stats['unique_customers']
    window_stats['revenue_per_customer'] = window_stats['total_revenue'] / window_stats['unique_customers']
    window_stats['customer_type'] = customer_type
    
    return window_stats

churned_activity = analyze_activity_windows(df, churned_customers, 'Churned')
active_activity = analyze_activity_windows(df, active_customers, 'Active')

activity_comparison = pd.concat([churned_activity, active_activity])

print("\n📊 ACTIVITY WINDOWS COMPARISON")
print("="*70)
print(activity_comparison.to_string(index=False))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

window_order = ['90+ days before', '61-90 days before', '31-60 days before', 'Last 30 days']

for customer_type, color in [('Churned', '#e74c3c'), ('Active', '#2ecc71')]:
    data = activity_comparison[activity_comparison['customer_type'] == customer_type]
    data = data.set_index('time_window').reindex(window_order)
    axes[0].plot(range(len(window_order)), data['orders_per_customer'].values, 
                 marker='o', label=customer_type, color=color, linewidth=2, markersize=8)

axes[0].set_xticks(range(len(window_order)))
axes[0].set_xticklabels(['90+ days\nbefore', '61-90 days\nbefore', '31-60 days\nbefore', 'Last 30\ndays'])
axes[0].set_ylabel('Orders per Customer')
axes[0].set_title('Purchase Frequency Over Time', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

for customer_type, color in [('Churned', '#e74c3c'), ('Active', '#2ecc71')]:
    data = activity_comparison[activity_comparison['customer_type'] == customer_type]
    data = data.set_index('time_window').reindex(window_order)
    axes[1].plot(range(len(window_order)), data['revenue_per_customer'].values, 
                 marker='s', label=customer_type, color=color, linewidth=2, markersize=8)

axes[1].set_xticks(range(len(window_order)))
axes[1].set_xticklabels(['90+ days\nbefore', '61-90 days\nbefore', '31-60 days\nbefore', 'Last 30\ndays'])
axes[1].set_ylabel('Revenue per Customer (£)')
axes[1].set_title('Spending Over Time', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 INSIGHTS:")
print("   → Active customers show INCREASING activity toward recent periods")
print("   → Churned customers show FLAT or DECLINING activity")
print("   → Activity trend is a strong predictor of churn!")

---

## 3. How Does Purchase Frequency Differ?

Comparing behavioral metrics between churned and active customers.

In [ ]:
customer_analysis = customer_summary_filtered.copy()
customer_analysis['status'] = customer_analysis['is_churned'].map({0: 'Active', 1: 'Churned'})

customer_analysis['avg_order_value'] = customer_analysis['total_revenue'] / customer_analysis['total_orders']
customer_analysis['orders_per_month'] = customer_analysis['total_orders'] / (customer_analysis['tenure_days'].clip(lower=1) / 30)

metrics_comparison = customer_analysis.groupby('status').agg(
    customers=('Customer ID', 'count'),
    avg_orders=('total_orders', 'mean'),
    median_orders=('total_orders', 'median'),
    avg_revenue=('total_revenue', 'mean'),
    avg_order_value=('avg_order_value', 'mean'),
    avg_tenure_days=('tenure_days', 'mean'),
    avg_orders_per_month=('orders_per_month', 'mean')
).round(2)

print("\n📊 CHURNED vs ACTIVE CUSTOMER COMPARISON")
print("="*70)
print(metrics_comparison.T)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for status, color in [('Active', '#2ecc71'), ('Churned', '#e74c3c')]:
    data = customer_analysis[customer_analysis['status'] == status]['total_orders']
    axes[0, 0].hist(data.clip(upper=20), bins=20, alpha=0.6, label=status, color=color, density=True)
axes[0, 0].set_xlabel('Number of Orders')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Order Count Distribution', fontsize=12, fontweight='bold')
axes[0, 0].legend()

for status, color in [('Active', '#2ecc71'), ('Churned', '#e74c3c')]:
    data = customer_analysis[customer_analysis['status'] == status]['total_revenue']
    axes[0, 1].hist(data.clip(upper=2000), bins=30, alpha=0.6, label=status, color=color, density=True)
axes[0, 1].set_xlabel('Total Revenue (£)')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('Revenue Distribution', fontsize=12, fontweight='bold')
axes[0, 1].legend()

for status, color in [('Active', '#2ecc71'), ('Churned', '#e74c3c')]:
    data = customer_analysis[customer_analysis['status'] == status]['avg_order_value']
    axes[1, 0].hist(data.clip(upper=500), bins=30, alpha=0.6, label=status, color=color, density=True)
axes[1, 0].set_xlabel('Average Order Value (£)')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title('AOV Distribution', fontsize=12, fontweight='bold')
axes[1, 0].legend()

for status, color in [('Active', '#2ecc71'), ('Churned', '#e74c3c')]:
    data = customer_analysis[customer_analysis['status'] == status]['tenure_days']
    axes[1, 1].hist(data, bins=30, alpha=0.6, label=status, color=color, density=True)
axes[1, 1].set_xlabel('Customer Tenure (days)')
axes[1, 1].set_ylabel('Density')
axes[1, 1].set_title('Tenure Distribution', fontsize=12, fontweight='bold')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

In [ ]:
from scipy import stats

print("\n📊 STATISTICAL SIGNIFICANCE TESTS")
print("="*60)

metrics_to_test = ['total_orders', 'total_revenue', 'avg_order_value', 'tenure_days']

for metric in metrics_to_test:
    churned_data = customer_analysis[customer_analysis['status'] == 'Churned'][metric].dropna()
    active_data = customer_analysis[customer_analysis['status'] == 'Active'][metric].dropna()
    
    stat, p_value = stats.mannwhitneyu(churned_data, active_data, alternative='two-sided')
    
    significance = "✅ SIGNIFICANT" if p_value < 0.05 else "❌ Not significant"
    print(f"\n{metric}:")
    print(f"   Churned mean: {churned_data.mean():.2f}, Active mean: {active_data.mean():.2f}")
    print(f"   p-value: {p_value:.4e} → {significance}")

---

## 4. What is Average Time-to-Churn?

Understanding when customers typically churn after their first/last purchase.

In [ ]:
churned_analysis = customer_analysis[customer_analysis['status'] == 'Churned'].copy()

churned_analysis['days_since_last_purchase'] = (churn_cutoff - churned_analysis['last_purchase']).dt.days

print("\n📊 TIME-TO-CHURN ANALYSIS")
print("="*60)

print(f"\nCustomer Lifetime (days from first to last purchase):")
print(f"   Mean: {churned_analysis['tenure_days'].mean():.1f} days")
print(f"   Median: {churned_analysis['tenure_days'].median():.1f} days")
print(f"   Std: {churned_analysis['tenure_days'].std():.1f} days")

print(f"\nDays of Inactivity Before Churn Label:")
print(f"   Mean: {churned_analysis['days_since_last_purchase'].mean():.1f} days")
print(f"   Median: {churned_analysis['days_since_last_purchase'].median():.1f} days")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(churned_analysis['tenure_days'], bins=50, color='#e74c3c', alpha=0.7, edgecolor='black')
axes[0].axvline(x=churned_analysis['tenure_days'].median(), color='blue', linestyle='--', 
                label=f'Median: {churned_analysis["tenure_days"].median():.0f} days')
axes[0].axvline(x=churned_analysis['tenure_days'].mean(), color='green', linestyle='--', 
                label=f'Mean: {churned_analysis["tenure_days"].mean():.0f} days')
axes[0].set_xlabel('Customer Lifetime (days)')
axes[0].set_ylabel('Number of Customers')
axes[0].set_title('Lifetime Distribution of Churned Customers', fontsize=12, fontweight='bold')
axes[0].legend()

lifetime_buckets = pd.cut(
    churned_analysis['tenure_days'],
    bins=[0, 7, 30, 60, 90, 180, 365, float('inf')],
    labels=['0-7d', '8-30d', '31-60d', '61-90d', '91-180d', '181-365d', '365d+']
)
bucket_counts = lifetime_buckets.value_counts().sort_index()

bars = axes[1].bar(range(len(bucket_counts)), bucket_counts.values, color='#e74c3c', alpha=0.7)
axes[1].set_xticks(range(len(bucket_counts)))
axes[1].set_xticklabels(bucket_counts.index)
axes[1].set_xlabel('Lifetime Bucket')
axes[1].set_ylabel('Number of Churned Customers')
axes[1].set_title('When Do Customers Churn?', fontsize=12, fontweight='bold')

for bar, val in zip(bars, bucket_counts.values):
    pct = val / len(churned_analysis) * 100
    axes[1].annotate(f'{pct:.1f}%',
                     xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                     ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n💡 INSIGHT: A large portion of churned customers have very short lifetimes!")
print(f"   → {(churned_analysis['tenure_days'] <= 30).mean()*100:.1f}% churn within first 30 days")
print(f"   → Early engagement is CRITICAL for retention")

---

## 5. Which Behavioral Patterns Correlate with Churn?

Identifying the strongest predictors of churn behavior.

In [ ]:
df_obs = df[df['InvoiceDate'] <= OBSERVATION_DATE].copy()

customer_features = df_obs.groupby('Customer ID').agg(
    last_purchase=('InvoiceDate', 'max'),
    first_purchase=('InvoiceDate', 'min'),
    
    total_orders=('Invoice', 'nunique'),
    total_transactions=('Invoice', 'count'),
    unique_products=('StockCode', 'nunique'),
    
    total_quantity=('Quantity', 'sum'),
    avg_quantity=('Quantity', 'mean'),
    total_revenue=('Price', lambda x: (x * df_obs.loc[x.index, 'Quantity']).sum()),
    
    avg_items_per_transaction=('StockCode', lambda x: len(x) / df_obs.loc[x.index, 'Invoice'].nunique())
).reset_index()

customer_features['recency_days'] = (OBSERVATION_DATE - customer_features['last_purchase']).dt.days
customer_features['tenure_days'] = (customer_features['last_purchase'] - customer_features['first_purchase']).dt.days
customer_features['avg_order_value'] = customer_features['total_revenue'] / customer_features['total_orders']
customer_features['orders_per_month'] = customer_features['total_orders'] / (customer_features['tenure_days'].clip(lower=1) / 30)

customer_features['is_churned'] = (customer_features['recency_days'] > CHURN_WINDOW).astype(int)

customer_features = customer_features[
    customer_features['first_purchase'] <= (OBSERVATION_DATE - timedelta(days=CHURN_WINDOW))
]

print(f"Created features for {len(customer_features):,} customers")

In [ ]:
feature_cols = ['recency_days', 'tenure_days', 'total_orders', 'total_transactions',
                'unique_products', 'total_quantity', 'total_revenue', 'avg_order_value',
                'orders_per_month', 'avg_items_per_transaction']

correlations = customer_features[feature_cols + ['is_churned']].corr()['is_churned'].drop('is_churned')
correlations = correlations.sort_values(key=abs, ascending=False)

print("\n📊 FEATURE CORRELATIONS WITH CHURN")
print("="*50)
for feature, corr in correlations.items():
    direction = "↑ Higher = More Churn" if corr > 0 else "↓ Higher = Less Churn"
    print(f"   {feature:30} : {corr:+.4f}  {direction}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#e74c3c' if c > 0 else '#2ecc71' for c in correlations.values]
bars = ax.barh(range(len(correlations)), correlations.values, color=colors)
ax.set_yticks(range(len(correlations)))
ax.set_yticklabels(correlations.index)
ax.set_xlabel('Correlation with Churn')
ax.set_title('Feature Correlations with Churn', fontsize=14, fontweight='bold')
ax.axvline(x=0, color='black', linewidth=0.5)

for bar, val in zip(bars, correlations.values):
    x_pos = val + 0.01 if val >= 0 else val - 0.01
    ha = 'left' if val >= 0 else 'right'
    ax.annotate(f'{val:.3f}', xy=(x_pos, bar.get_y() + bar.get_height()/2),
                ha=ha, va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n💡 TOP CHURN PREDICTORS:")
print("   1. Recency (days since last purchase) - STRONGEST predictor")
print("   2. Total orders - More orders = Less likely to churn")
print("   3. Total revenue - Higher spenders are more loyal")
print("   4. Unique products - Product diversity indicates engagement")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

corr_matrix = customer_features[feature_cols + ['is_churned']].corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdYlGn_r',
            center=0, square=True, linewidths=0.5, ax=ax)
ax.set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

---

##  EDA Summary & Key Insights

In [ ]:
print("="*70)
print("           📊 EXPLORATORY DATA ANALYSIS SUMMARY")
print("="*70)

print(f"""
╔══════════════════════════════════════════════════════════════════════╗
║                     KEY FINDINGS                                     ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  1. CHURN RATE                                                       ║
║     → Overall churn rate: {customer_analysis['is_churned'].mean()*100:>5.1f}%                                ║
║     → One-time buyers: ~{churn_by_orders.loc['1 order', 'churn_rate_pct']:.0f}% churn (highest risk)                     ║
║     → 10+ order customers: ~{churn_by_orders.loc['10+ orders', 'churn_rate_pct']:.0f}% churn (most loyal)              ║
║                                                                      ║
║  2. ACTIVITY PATTERNS                                                ║
║     → Churned customers show flat/declining activity                 ║
║     → Active customers show increasing engagement                    ║
║     → Activity trend is a leading indicator of churn                 ║
║                                                                      ║
║  3. CUSTOMER LIFETIME                                                ║
║     → Median churned customer lifetime: {churned_analysis['tenure_days'].median():.0f} days                 ║
║     → {(churned_analysis['tenure_days'] <= 30).mean()*100:.0f}% of churned customers leave within 30 days          ║
║     → Early engagement is CRITICAL                                   ║
║                                                                      ║
║  4. TOP CHURN PREDICTORS                                             ║
║     1. Recency (days since last purchase)                            ║
║     2. Purchase frequency (total orders)                             ║
║     3. Customer value (total revenue)                                ║
║     4. Product engagement (unique products)                          ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
""")

In [ ]:
output_path = os.path.join(PROCESSED_DATA_PATH, 'customer_features_eda.parquet')
customer_features.to_parquet(output_path, index=False)
print(f"\n✅ Customer features saved to: {output_path}")

---

##  Phase 2 Checklist

- [x] Calculated overall churn rate (~X%)
- [x] Analyzed activity decline patterns before churn
- [x] Compared purchase frequency (churned vs active)
- [x] Calculated average time-to-churn
- [x] Identified behavioral patterns correlating with churn
- [x] Created SQL queries (saved in /sql/)
- [x] Generated visualizations with insights
- [x] Documented key findings

**Phase 2 Status: COMPLETE** 

---

**Next**: Phase 3 - Feature Engineering